In [ ]:
%matplotlib inline
from __future__ import print_function
import gc
import math
import os
import random
import sys
import traceback
import sklearn.metrics
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import ndimage

import outputer
import improc
import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (improc)
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

# Enumerate Images
Image names are sequential, so add every tenth image to the validation set based on filename.

In [ ]:
training = []
test = []

for root, dirs, files in os.walk('captures'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        # Find all the image files, split into test and training.
        if low_name.endswith(".png"):
            if low_name.endswith("0.png"):
                test.append(path)
            else:
                training.append(path)

print("Training:", len(training), "Test:", len(test))
print(training[:2])
print(test[:2])

# Image Processing
Each image file contains a color image (top half), and an encoded depth image (bottom half)
<img src="testing/IMG_2114.PNG">
* Note: The image may also contain the orientation data. If so it is encoded in the first two pixels of the depth image. If the first pixel of the depth image is red, the second has the x, y, z, w quaternion components encoded in the r,g,b,a values.

The improc module contains functions for splitting the image, decoding the depth back into floating point millimeters, and for filling in gaps.

In [ ]:
# Precomputed via compute_average_depth()
MEAN_DEPTH = np.float32(1688.97)

print(MEAN_DEPTH)

In [ ]:
depth_image_cache_path = outputer.setup_directory("temp", "cache")

class ImageSampler(object):
    """Wrap an image for sampling."""
    def __init__(self, image_file):
        # Process the image or grab it from the cache.
        # image is normalized CIELAB, depth is not normalized.
        self.image, self.depth = improc.process_cached(depth_image_cache_path, image_file)
        self.depth /= improc.MAX_DEPTH

    def sample(self, image_slot, depth_slot, height_offset=None, width_offset=None):
        height = image_slot.shape[0]
        spare_height = self.image.shape[0] - height
        y = spare_height / 2 if height_offset is None else height_offset
        
        width = image_slot.shape[1]
        spare_width = self.image.shape[1] - width
        x = spare_width / 2 if width_offset is None else width_offset
        
        image_slot[:,:,:] = self.image[y : y + height, x : x + width, : image_slot.shape[-1]]
        depth_slot[:,:,0] = self.depth[y : y + height, x : x + width]

In [ ]:
def prepare_images(image_paths, inputs, targets):
    for i, sampler in enumerate([ImageSampler(path) for path in image_paths]):
        sampler.sample(inputs[i], targets[i])

## Image processing examples

In [ ]:
example_image, example_depth, example_attitude = improc.load_image("testing/IMG_2114.PNG")
plt.imshow(example_image)
print(example_image.shape, example_image.dtype)

In [ ]:
plt.imshow(example_depth)
print(example_depth.shape, example_depth.dtype)
print(example_attitude)

In [ ]:
sampler = ImageSampler("testing/IMG_2114.PNG")
sample_size = 100
image_sample = np.zeros(shape=(sample_size, sample_size, 3))
depth_sample = np.zeros(shape=(sample_size, sample_size, 1))
sampler.sample(image_sample, depth_sample)

print(image_sample.shape, image_sample.dtype)
plt.imshow(image_sample)

In [ ]:
plt.imshow(depth_sample.reshape(sample_size, sample_size))
print(depth_sample.shape, depth_sample.dtype)
print(np.min(depth_sample), np.max(depth_sample))

# Data Management

In [ ]:
COLOR_CHANNELS = 3
image_height = 480
image_width = 640

data_files = {
    "image_size": (image_height, image_width, COLOR_CHANNELS),
    "depth_size": (image_height, image_width, 1),
    "train_files": np.array(sorted(training)),
    "test_files": np.array(sorted(test))
}

del training
del test

In [ ]:
def setup_cross_validation(data, train_count, valid_count, test_count=None, chunk_size=None, entropy=random):
    cross_data = data.copy()
    
    if chunk_size:
        cross_data["image_size"] = chunk_size
        cross_data["depth_size"] = chunk_size[:-1] + (1,)

    paths = cross_data["train_files"][:]
    mutate.fisher_yates_shuffle(paths, entropy)

    cross_data["train_files"] = paths[:train_count]
    cross_data["valid_files"] = paths[train_count:train_count + valid_count]
    
    if test_count is not None:
        cross_data["test_files"] = data["test_files"][:test_count]
    
    return cross_data

# Graph Setup

In [ ]:
def setup_graph(
    batch_size,
    image_shape,
    target_shape,
    layer_stack
):
    graph = tf.Graph()
    with graph.as_default():
        input_shape = (batch_size,) + image_shape
        output_shape = (batch_size,) + target_shape
        train   = tf.placeholder(tf.float32, shape=input_shape)
        targets = tf.placeholder(tf.float32, shape=output_shape)
        verify  = tf.placeholder(tf.float32, shape=input_shape)

        layers = layer_stack.construct(input_shape, output_shape)
        l2_loss = convnet.setup_layers(layers)

        results = convnet.connect_model(train, layers, True)[-1]
        
        # Fill NaNs in target with values from results to
        # eliminate any contribution to the gradient
        valid_targets = tf.select(tf.is_nan(targets), results, targets)
        
        loss = tf.reduce_mean(tf.squared_difference(results, valid_targets)) + l2_loss
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "targets": targets,
            "loss": loss,
            "optimizer": layer_stack.construct_optimizer(loss),

            # Predictions for training and verification (validation or test)
            "predictions": results,
            "verify": verify,
            "verify_predictions": convnet.connect_model(verify, layers, False)[-1]
        }
    return info

# Graph Execution

In [ ]:
def prediction_error(predictions, targets):
    is_finite = np.isfinite(targets)
    where_valid = np.where(is_finite)
    error = sklearn.metrics.mean_squared_error(predictions[where_valid], targets[where_valid])
    return error, np.count_nonzero(is_finite)

In [ ]:
def make_predictor(session, graph_info):
    def predict(inputs, targets):
        feed_dict = {graph_info["verify"]: inputs}
        return session.run([graph_info["verify_predictions"]], feed_dict=feed_dict)[0]
    return predict

In [ ]:
def batch_error(predictor, files, inputs, targets, batch_size):
    total_error = 0
    total_count = 0
    batch_count = len(files) / batch_size
    for b in xrange(batch_count):
        offset = b * batch_size
        end = offset + batch_size
        prepare_images(files[offset:end], inputs, targets)
        predictions = predictor(inputs, targets)
        error, count = prediction_error(predictions, targets)
        total_error += error
        total_count += count
    return (total_error / np.float32(total_count)), predictions[-1], targets[1]

In [ ]:
def depth_mean_like(depths):
    return np.ones_like(depths) * (MEAN_DEPTH / improc.MAX_DEPTH)

def always_guess_mean_error(files, inputs, targets, batch_size):
    def predict_mean(images, depths):
        return depth_mean_like(depths)
    return batch_error(predict_mean, files, inputs, targets, batch_size)[0]

In [ ]:
def run_graph(
    graph_info,
    data,
    step_count,
    report_every=50,
    verbose=True,
    compute_test=False,
    error_maximum=None
):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        height, width, channels = data["image_size"]
        valid_error = 1
        predictor = make_predictor(session, graph_info)

        training_files = data["train_files"]
        
        batch_inputs = np.empty(shape=(batch_size, height, width, channels), dtype=np.float32)
        batch_targets = np.empty(shape=(batch_size, height, width, 1), dtype=np.float32)

        guess_mean_error = always_guess_mean_error(
            data["valid_files"], batch_inputs, batch_targets, batch_size
        )
        print("Error guessing mean:", guess_mean_error)
        
        for step in xrange(step_count + 1):
            # Pick an offset within the training data, which has been randomized.
            offset = (step * batch_size) % (training_files.shape[0] - batch_size)
            # Generate a minibatch.
            batch_files = training_files[offset:(offset + batch_size)]
            prepare_images(batch_files, batch_inputs, batch_targets)
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            targets = [
                graph_info["optimizer"],
                graph_info["loss"],
                graph_info["predictions"]
            ]
            feed_dict = {
                graph_info["train"] : batch_inputs,
                graph_info["targets"] : batch_targets
            }
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if np.isnan(l):
                print("Error computing loss:", l)
                print(np.sum(np.isnan(predictions)))
                return 0, None
            if (step % report_every == 0):
                if verbose:
                    print("Minibatch loss at step", step, ":", l)
                    print("Minibatch error:", prediction_error(predictions, batch_targets))
                valid_error, _, _ = batch_error(
                    predictor, data["valid_files"],
                    batch_inputs, batch_targets, batch_size
                )
                print("Validation error:", valid_error)
                if error_maximum and step > 0 and valid_error < error_maximum:
                    print("Early out.")
                    break
        results = (predictions[0], batch_targets[0], batch_inputs[0])
        if compute_test:
            test_results = batch_error(
                predictor, data["test_files"],
                batch_inputs, batch_targets, batch_size
            )
            print("Test error:", test_results[0])
            results = results + test_results
        return guess_mean_error - min(valid_error, 1), results

## Testing Error Metrics

In [ ]:
TEST_BATCH = 1
test_inputs = np.empty(shape=(TEST_BATCH, 480, 640, COLOR_CHANNELS), dtype=np.float32)
test_depths = np.empty_like(test_inputs[:,:,:,:1])
prepare_images(proto_cross["test_files"][:TEST_BATCH], test_inputs, test_depths)

In [ ]:
plt.imshow(test_inputs[0,:,:,0], cmap='Greys_r')
print(np.min(test_inputs[0,:,:,0]),np.max(test_inputs[0,:,:,0]))
print(np.min(test_inputs[0,:,:,1]),np.max(test_inputs[0,:,:,1]))
print(np.min(test_inputs[0,:,:,2]),np.max(test_inputs[0,:,:,2]))

In [ ]:
plt.imshow(test_depths[0,:,:,0])
depths_valid = np.where(np.isfinite(test_depths[0]))
print(np.min(test_depths[0][depths_valid]),np.max(test_depths[0][depths_valid]))

In [ ]:
print(test_inputs.shape)
print(test_depths.shape)
prediction_error(test_inputs[:,:,:,0:1], test_depths)

In [ ]:
prediction_error(np.zeros_like(test_depths), test_depths)

In [ ]:
prediction_error(depth_mean_like(test_depths), test_depths)

## Testing components

In [ ]:
batch_size = 4
sample_size = data_files["image_size"]
depth_size = data_files["depth_size"]
proto_cross = setup_cross_validation(data_files, 9800, 200, 1000, sample_size)
conv_layers = [
    ("conv",       5, 2, 10, "SAME", False),
    ("conv",      10, 2, 20, "SAME", False),
    ("conv_bias", 15, 5, 25, "SAME", False)
]
expand_layers = [
    (5, 5, "SAME", True, False),
    (2, 5, "SAME", True, False),
    (2, 5, "SAME", True, False)
]
prototype = convevo.create_stack(conv_layers, expand_layers, False, [], 0.0, 0.01, 0.0)
prototype.make_safe((batch_size,) + sample_size, (batch_size,) + depth_size)
prototype.reseed(random.Random(24601))
prototype_graph = setup_graph(batch_size, sample_size, depth_size, prototype)

In [ ]:
score, results = run_graph(prototype_graph, proto_cross, 8, 4, True)
print(score)

In [ ]:
plt.imshow(results[0][:,:,0])
print(np.min(results[0]),np.max(results[0]))

In [ ]:
plt.imshow(results[1].reshape(sample_size[0],sample_size[1]))

In [ ]:
plt.imshow(results[2])